Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 02: Bias potential and metadynamics (MetaD)

In [1]:
from mindspore import context
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units
from sponge import Protein
from sponge import WithEnergyCell
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.control import VelocityVerlet
from sponge.callback import WriteH5MD, RunInfo
from sponge.control import Langevin
from sponge.sampling import Metadynamics
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion
from sponge.function import PI

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-18 15:38:44
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.40555, phi: -3.1340268, psi: -3.1368413
[MindSPONGE] Step: 20, E_pot: -60.168476, phi: -3.1277595, psi: -3.1337605
[MindSPONGE] Step: 30, E_pot: -61.549706, phi: -3.1221528, psi: -3.131402
[MindSPONGE] Step: 40, E_pot: -62.696198, phi: -3.1171532, psi: -3.1295478
[MindSPONGE] Step: 50, E_pot: -63.674084, phi: -3.112676, psi: -3.1280725
[MindSPONGE] Step: 60, E_pot: -64.52254, phi: -3.1086385, psi: -3.1268952
[MindSPONGE] Step: 70, E_pot: -65.268196, phi: -3.1049676, psi: -3.1259575
[MindSPONGE] Step: 80, E_pot: -65.93031, phi: -3.1016028, psi: -3.1252148
[MindSPONGE] Step: 90, E_pot: -66.52346, phi: -3.0984938, psi: -3.1246324
[MindSPONGE] Finished simulation at 2023-05-18 15:38:48
[MindSPONGE] Simulation time: 3.72 seconds.
--------------------------------------------------------------------------------


In [7]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

[WARNING] PRE_ACT(1334564,7fb2dc248500,python):2023-05-18-15:38:51.435.339 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:255] CalMemBlockAllocSize] Memory not enough: current free memory size[5308416] is smaller than required size[299635200].


In [8]:
sim = WithEnergyCell(system, potential, bias=metad)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [10]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a02.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-18 15:38:55
[MindSPONGE] Step: 0, E_pot: -67.058876, E_kin: 0.0, E_tot: -67.058876, Temperature: 0.0, phi: -3.0957475, psi: -3.1241677
[MindSPONGE] Step: 10, E_pot: 663.6504, E_kin: 708.6173, E_tot: 1372.2677, Temperature: 2705.6213, phi: -3.0849383, psi: -3.1222496
[MindSPONGE] Step: 20, E_pot: 1978.3878, E_kin: 2713.9036, E_tot: 4692.2915, Temperature: 10362.1455, phi: -3.135781, psi: 2.9905124
[MindSPONGE] Step: 30, E_pot: 2066.9265, E_kin: 2190.066, E_tot: 4256.992, Temperature: 8362.044, phi: 3.125073, psi: 2.7188087
[MindSPONGE] Step: 40, E_pot: 2583.5564, E_kin: 2501.6257, E_tot: 5085.182, Temperature: 9551.632, phi: -2.9243853, psi: 2.4907224
[MindSPONGE] Step: 50, E_pot: 113907310000.0, E_kin: 21849186.0, E_tot: 113929160000.0, Temperature: 83423900.0, phi: -0.17568907, psi: 2.0777006
[MindSPONGE] Step: 60, E_pot: nan, E_kin: nan, E_tot: nan, Temperature: nan, phi: nan, psi: nan
[MindSPONGE] Step: 70, E_pot: nan, E_kin: nan, E_tot: na